In [2]:
import numpy as np
from scipy.optimize import minimize


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [3]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
#res = minimize(rosen, x0, method='nelder-mead',
#               options={'xatol': 1e-8, 'disp': True})

In [9]:
x0[0]

1.3

In [4]:
#acoustic values: ax1, ax2, ay1, ay2, kx1, kx2, ky1, ky2
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0])

array([1., 1., 1., 1., 1.])